In [1]:
from bokeh.plotting import output_notebook
output_notebook()
import pandas as pd
import numpy as np
import geopandas as gpd

from bokeh.plotting import figure, show, save, ColumnDataSource
from bokeh.models import ColumnDataSource, LabelSet, Label,HoverTool,LinearColorMapper,ColumnDataSource
from bokeh.models import LinearInterpolator,  ColorBar,CDSView, GroupFilter
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
from bokeh.models import LogColorMapper, LogTicker, ColorBar
from bokeh.layouts import row

from bokeh.io import output_file
from bokeh.models import FixedTicker,FuncTickFormatter,NumeralTickFormatter,PrintfTickFormatter, BasicTicker,LinearColorMapper

Loading BokehJS ...

In [2]:
# Output inline in the notebook
output_file('Legend.html',
            title='Price per year')

In [3]:
trainlines = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='railway')
clyde = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='river')
#stations  contains schematics geometry and calculated price
stations = gpd.read_file(r'd:\viz\trainlines\derived\New folder\station_price.gpkg', layer='price_dz_points')
station_labels =gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='railstation_labels')
stations['mean'] = pd.to_numeric(stations['mean']).round()
station_labels['name'] = station_labels['name'].str.upper()

In [4]:
#stations.head()

In [5]:
subway  = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='subway')
subway_labels  = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='subway_lables')
subway_labels['name'] = subway_labels['name'].str.upper()

In [6]:
tsource = ColumnDataSource(trainlines)
stsource = ColumnDataSource(stations)
ssource = ColumnDataSource(subway)
slsource = ColumnDataSource(subway_labels)
stlsource = ColumnDataSource(station_labels)

In [7]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [8]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [9]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [10]:
# Get the Polygon x and y coordinates
#grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
#grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

# Calculate x and y coordinates of the line
trainlines['x'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
trainlines['y'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)


clyde['x'] = clyde.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
clyde['y'] = clyde.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

In [11]:
# Calculate x and y coordinates of the points
subway['x'] = subway.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
subway['y'] = subway.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

subway_labels['x'] = subway_labels.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
subway_labels['y'] = subway_labels.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

station_labels['x'] = station_labels.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
station_labels['y'] = station_labels.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [12]:
# Store the data in a ColumnDataSource
stations['year'] = stations['year'].astype('str')
#stations_s = ColumnDataSource(stations.drop(columns=['geometry','fclass','time_man']))

#stations_s.data.keys()

In [13]:
# Make a copy, drop the geometry column and create ColumnDataSource
tr_df = trainlines.drop('geometry', axis=1).copy()
trsource = ColumnDataSource(tr_df)

# Make a copy, drop the geometry column and create ColumnDataSource
cl_df = clyde.drop('geometry', axis=1).copy()
clsource = ColumnDataSource(cl_df)


# Make a copy, drop the geometry column and create ColumnDataSource
s_df = subway.drop('geometry', axis=1).copy()
ssource = ColumnDataSource(s_df)

# Make a copy, drop the geometry column and create ColumnDataSource
sl_df = subway_labels.drop('geometry', axis=1).copy()
slsource = ColumnDataSource(sl_df)


# Make a copy, drop the geometry column and create ColumnDataSource
st_df = stations.drop('geometry', axis=1).copy()
stsource = ColumnDataSource(st_df)

# Make a copy, drop the geometry column and create ColumnDataSource
stl_df = station_labels.drop('geometry', axis=1).copy()
stlsource = ColumnDataSource(stl_df)

In [14]:
p3 = figure(plot_width=1150, plot_height=800, title='Click Legend to HIDE Data',
          toolbar_location=None)

In [15]:
# Add clyde on top of the same figure
p3.multi_line('x', 'y', source=clsource, color="#5e96cb", line_width=14, line_cap ='round', line_join = 'round')
# Add clyde additional styling 
##TO-DO find a better way to achieve similar styling result
p3.multi_line('x', 'y', source=clsource, color="white", line_width=12, line_cap ='round', line_join = 'round')
p3.multi_line('x', 'y', source=clsource, color="#98bcde", line_width=10, line_cap ='round', line_join = 'round')

GlyphRenderer(id='1060', ...)

In [16]:
# Add trainlines on top of the same figure
p3.multi_line('x', 'y', source=trsource, color="#B0B0B0", line_width=7, line_cap ='round', line_join = 'round')
#just for styling
p3.multi_line('x', 'y', source=trsource, color="white", line_width=5, line_cap ='round', line_join = 'round')
p3.multi_line('x', 'y', source=trsource, color="#B0B0B0", line_width=3, line_cap ='round', line_join = 'round')

GlyphRenderer(id='1077', ...)

In [17]:
# Add subway on top (as black points)
subway = p3.circle('x', 'y', size=10, source=ssource, color="pink", fill_alpha = 0.5)
# Add stations on top (as blue hollow points)

#stations = p3.circle(x="x", y="y",size=1, source=stsource,fill_alpha=0.1,color = '#0e3271')

In [18]:
#labels for subway
labels2 = LabelSet(x='x', y='y', text='name',
                   text_font ='calibri',text_font_style ='italic', text_font_size='8pt',text_color='#0e3271',
                   x_offset=-5,
                  # level='glyph', 
                   source=slsource,render_mode='canvas')
#  angle = 30,angle_units='deg',x_offset=-15, y_offset=10,y_offset=10, x_offset=-15,
#lables for stations
labels1 = LabelSet(x='x', y='y',x_offset= -5, y_offset= 10,
                   text_font ='calibri',text_font_style ='italic',text='name',text_font_size='8pt',text_color='#696969',
                  # level='glyph',#angle = 30,angle_units='deg',
               source=stlsource,render_mode='canvas')

In [19]:
p3.add_layout(labels1)
p3.add_layout(labels2)

In [20]:
#removing grid lines

p3.xgrid.grid_line_color = None
p3.ygrid.grid_line_color = None

#hiding axis
p3.axis.visible = False

In [21]:
# Hover tool referring to our own data field using @ and
## a position on the graph using $
# adding hover only to subway and railway stations
hover = HoverTool( tooltips = [('Year', '@year'),('Price','@mean')]) #,('(x,y)', '($x, $y)')])
p3.add_tools(hover)


In [22]:
#p.hover.renderers=[subway, stations]


In [23]:
# Create a view for each year
filters_2008 = [GroupFilter(column_name='year', group='2008')]
view_2008 = CDSView(source=stsource, filters=filters_2008)

filters_2009 = [GroupFilter(column_name='year', group='2009')]
view_2009 = CDSView(source=stsource,filters=filters_2009)

In [24]:
#for key in stations_s.data.keys():
   # for value in stations_s.data[key]:
       # print (key,value, type(value))

In [25]:
# Create a view for each year
filters_2008  = [GroupFilter(column_name='year', group='2008')]
view_2008 = CDSView(source=stsource,
                      filters=filters_2008 )

filters_2009 = [GroupFilter(column_name='year', group='2009')]
view_2009 = CDSView(source=stsource,
                      filters=filters_2009)
filters_2010  = [GroupFilter(column_name='year', group='2010')]
view_2010 = CDSView(source=stsource,
                      filters=filters_2010 )

filters_2011 = [GroupFilter(column_name='year', group='2011')]
view_2011 = CDSView(source=stsource,
                      filters=filters_2011)
filters_2012  = [GroupFilter(column_name='year', group='2012')]
view_2012 = CDSView(source=stsource,
                      filters=filters_2012 )

filters_2013 = [GroupFilter(column_name='year', group='2013')]
view_2013 = CDSView(source=stsource,
                      filters=filters_2013)
filters_2014  = [GroupFilter(column_name='year', group='2014')]
view_2014 = CDSView(source=stsource,
                      filters=filters_2014 )

filters_2015 = [GroupFilter(column_name='year', group='2015')]
view_2015 = CDSView(source=stsource,
                      filters=filters_2015)
filters_2016 = [GroupFilter(column_name='year', group='2016')]
view_2016 = CDSView(source=stsource,
                      filters=filters_2016)

filters_2017 = [GroupFilter(column_name='year', group='2017')]
view_2017 = CDSView(source=stsource,
                      filters=filters_2017)


filters_2018 = [GroupFilter(column_name='year', group='2018')]
view_2018 = CDSView(source=stsource,
                      filters=filters_2018)

In [26]:
size_mapper=LinearInterpolator(
    x=[min(stsource.data['mean']),max(stsource.data['mean'])],
    y=[5,50]
)

In [27]:
#colormap = LinearColorMapper(palette='Magma256', low=min(stsource.data['mean']), high=max(stsource.data['mean']))
colormap1 = LinearColorMapper(palette='Viridis256' , low=min(stsource.data['mean']), high=max(stsource.data['mean']))
colormap2 = linear_cmap(field_name='mean',palette='Viridis256' , low=min(stsource.data['mean']), high=max(stsource.data['mean']))

In [28]:
common_circle_kwargs = {
    'x': 'x',
    'y': 'y',
    'source': stsource,
    'color':colormap2,
    'size':{'field':'mean','transform': size_mapper},
    'alpha': 0.9,
}


In [29]:
common_2008_kwargs = {
    'view': view_2008,
    #'color': '#002859',
    'legend': '2008'
}
common_2009_kwargs = {
    'view': view_2009,
    #'color': '#FFC324',
    'legend': '2009'
}
common_2010_kwargs = {
    'view': view_2010,
    #'color': 'red',
    'legend': '2010'
}
common_2011_kwargs = {
    'view': view_2011,
   # 'color': 'orange',
    'legend': '2011'
}
common_2012_kwargs = {
    'view': view_2012,
    #'color': '#FFC324',
    'legend': '2012'
}
common_2013_kwargs = {
    'view': view_2013,
   # 'color': 'lime',
    'legend': '2013'
}
common_2014_kwargs = {
    'view': view_2014,
    #'color': 'black',
    'legend': '2014'
}
common_2015_kwargs = {
    'view': view_2015,
    #'color': 'grey',
    'legend': '2015'
}
common_2016_kwargs = {
    'view': view_2016,
    #'color': 'purple',
    'legend': '2016'
}
common_2017_kwargs = {
    'view': view_2017,
    #'color': 'brown',
    'legend': '2017'
}
common_2018_kwargs = {
    'view': view_2018,
    #'color': 'pink',
    'legend': '2018'
}

In [30]:
# Create the two figures and draw the data
#p3 = figure(**common_figure_kwargs,                   title='Click Legend to HIDE Data')
c1 = p3.circle(**common_circle_kwargs, **common_2008_kwargs)
c2 =p3.circle(**common_circle_kwargs, **common_2009_kwargs)
c3 =p3.circle(**common_circle_kwargs, **common_2010_kwargs)
c4 =p3.circle(**common_circle_kwargs, **common_2011_kwargs)
c5 =p3.circle(**common_circle_kwargs, **common_2012_kwargs)
c6=p3.circle(**common_circle_kwargs, **common_2013_kwargs)
c7 =p3.circle(**common_circle_kwargs, **common_2014_kwargs)
c8 =p3.circle(**common_circle_kwargs, **common_2015_kwargs)
c9=p3.circle(**common_circle_kwargs, **common_2016_kwargs)
c10=p3.circle(**common_circle_kwargs, **common_2017_kwargs)
c11=p3.circle(**common_circle_kwargs, **common_2018_kwargs)

In [32]:
p3.hover.renderers=[c1, c2,c3,c4,c5,c6,c7,c8,c9,c10,c11]

In [33]:
tick_labels = {'100000':'100K','150000':'150K','200000':'200K','250000':'250K','300000':'300K','350000':'350K','500000':'500K','750000':'750K','950000':'950K'}

In [34]:
color_bar = ColorBar(color_mapper=colormap1, width=8,major_label_overrides=tick_labels,
                     #title ='Price, £',title_text_align = 'center',
                     label_standoff=15, border_line_color=None,location=(-10,0))

In [35]:
p3.add_layout(color_bar, 'right')

In [36]:
# Add interactivity to the legend
p3.legend.click_policy = 'hide'
show(p3)